In [27]:
import pandas as pd
from sklearn.model_selection import train_test_split
import tensorflow as tf
from tensorflow.keras import layers, models
import tensorflow.keras
import autokeras as ak
import keras
import numpy as np

In [28]:
# Load training dataset
df1 = pd.read_csv('./bioresponse/train.csv')
#df1.describe() # Visualize

# Load testing dataset
kaggle_test = pd.read_csv('./bioresponse/test.csv')
# test.describe() # Visualize

# Separate labels from training dataset for fitting
labels = df1['Activity']
del df1['Activity']

# 80/20 train test split
x_train, x_test, y_train, y_test = train_test_split(df1, labels, test_size=0.2)
input_shape = x_train.shape
print("Shape of Input Data: " + str(input_shape))
print("Shape of Test Labels: " + str(y_test.shape))

Shape of Input Data: (3000, 1776)
Shape of Test Labels: (751,)


In [34]:
model = models.Sequential()
model.add(layers.Dense(128, input_dim=x_train.shape[1], activation='relu'))
model.add(layers.Dense(64, activation='relu'))
model.add(layers.Dense(1, activation='sigmoid'))
model.summary()

Model: "sequential_26"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_34 (Dense)            (None, 128)               227456    
                                                                 
 dense_35 (Dense)            (None, 64)                8256      
                                                                 
 dense_36 (Dense)            (None, 1)                 65        
                                                                 
Total params: 235777 (921.00 KB)
Trainable params: 235777 (921.00 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [35]:
model.compile(
    optimizer="adam",
    loss=tf.keras.losses.BinaryCrossentropy(),
    metrics=["accuracy"],
)

history = model.fit(
    x_train, y_train, epochs=10, validation_data=(x_test, y_test)
)

Epoch 1/10
94/94 [==============================] - 2s 11ms/step - loss: 0.6072 - accuracy: 0.6737 - val_loss: 0.5077 - val_accuracy: 0.7603
Epoch 2/10
94/94 [==============================] - 1s 9ms/step - loss: 0.4921 - accuracy: 0.7630 - val_loss: 0.4817 - val_accuracy: 0.7790
Epoch 3/10
94/94 [==============================] - 1s 9ms/step - loss: 0.4711 - accuracy: 0.7753 - val_loss: 0.4865 - val_accuracy: 0.7856
Epoch 4/10
94/94 [==============================] - 1s 8ms/step - loss: 0.4556 - accuracy: 0.7843 - val_loss: 0.5633 - val_accuracy: 0.7377
Epoch 5/10
94/94 [==============================] - 1s 8ms/step - loss: 0.4550 - accuracy: 0.7927 - val_loss: 0.4925 - val_accuracy: 0.7856
Epoch 6/10
94/94 [==============================] - 1s 8ms/step - loss: 0.4267 - accuracy: 0.8050 - val_loss: 0.5538 - val_accuracy: 0.7670
Epoch 7/10
94/94 [==============================] - 1s 7ms/step - loss: 0.4077 - accuracy: 0.8173 - val_loss: 0.4849 - val_accuracy: 0.8016
Epoch 8/10
94/94 [=

In [49]:
prob_output = model.predict(kaggle_test)
print(prob_output)
print(prob_output.shape)
#arr = prob_output.reshape(1,2501)
df = pd.DataFrame(data=prob_output, columns=['PredictedProbability'])
df['MoleculeId'] = df.index + 1
df.head()
df.to_csv('submissionDL1.csv',index=False)

79/79 [==============================] - 0s 2ms/step
[[0.999969  ]
 [0.9956514 ]
 [0.71711004]
 ...
 [0.99346477]
 [0.9642539 ]
 [0.8692305 ]]
(2501, 1)
